# Charger les bibs

In [ ]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras import optimizers
from PIL import Image
import numpy as np
import glob
import os


# Paramétres de l'inférence

In [ ]:
model_path = './Modelcpp1.h5'
img_path = 'C:/Users/adm_ael-filou/Desktop/Exemple_Anis/prediction/2.png'
folder_path ='./all/'
folder_new_img_path = "./prediction/"
pred =['Compound Switch','Oblique Crossing','Single Switch']


# Charger et compiler le modéle

In [ ]:
def load_and_compile(model_path):
    
    # load the trained model
    opt = optimizers.Adam(learning_rate =0.001)
    model = load_model(model_path)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

# Pré traitement des images

In [ ]:
def pre_teret2(img,min_size=256, fill_color=(0, 0, 0)):
    
    old_size = img.size  # old_size[0] is in (width, height) format
    ratio = float(min_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # thumbnail is a in-place operation
    img = img.resize(new_size)
    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (min_size, min_size))
    new_im.paste(img, ((min_size-new_size[0])//2,
                        (min_size-new_size[1])//2))
        
    return new_im

In [ ]:
model = load_and_compile(model_path)

# Inférer une seule image

In [ ]:
def infer_one_img(model,pred,img_path):
    test_image = pre_teret2(Image.open(img_path))
    #test_image.show()
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    return(pred[np.argmax(result)])
res = infer_one_img(model,pred,img_path)
print(res)

# Inférer les dossiers test

In [ ]:
def infer_test_folders(model,pred,folder_path):
    for i in pred:
        print(f"-------les images du dossier {i} --------------")
        for file in glob.iglob(os.path.join(folder_path+i,"*.png")):
            test_image = pre_teret2(Image.open(file))
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = model.predict(test_image)
            print(f"here the result {result} and the pred {pred[np.argmax(result)]}")
            if (pred[np.argmax(result)] != i):
                print(file)
                print(f"fausse préduction p = {pred[np.argmax(result)]} GT = {i}")
                print(result)
                print("----------------------------------------------------------")
infer_test_folders(model,pred,folder_path)

In [ ]:
['Compound Switch','Oblique Crossing','Single Switch']

# Inférer les nouvelles images

In [ ]:
def infer_new_imgs(model,pred,folder_new_img_path):
    for file in glob.iglob(os.path.join(folder_new_img_path,"*.png")):
        test_image = pre_teret2(Image.open(file))
        #test_image.show()
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        print("----------------------------------------------------------")
        print(file)
        print(f"here the result {result} and the pred {pred[np.argmax(result)]}")
infer_new_imgs(model,pred,folder_new_img_path)